In [ ]:
## unimportant, makes the notebook deterministic

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

import numpy
import random
random.seed(0)

import numpy as np
np.random.seed(0)

# Active Learning:

## Motivation
A key problem in deep learning is data efficiency. While excellent performance can be obtained with modern tools, these are often data-hungry, rendering the deployment of deep learning in the real-world challenging for many tasks.  In Active Learning we use a “human in the loop” approach to data labelling, reducing the amount of data that needs to be labelled drastically, and making machine learning applicable when labelling costs would be too high.





## Passive Learning
Tasks which involve gathering a large amount of data randomly sampled from the underlying distribution and using this large dataset to train a model that can perform some sort of prediction. This method has many disadvantages:
* Too many wasted samples. 
* Learning is limited by sampling resolution 

<img src=https://imgur.com/M7Afc39.png width="400">

## Active Learning: Definition and Concepts
Active Learning is a semi-supervised technique whose main hypothesis is that if a learning algorithm can select the data it wants to learn from, it can perform better than traditional methods with significantly less data for training. In other words, **Active Learning (AL) is an interactive approach to simultaneously build a labelled data set and train a machine learning model.**

**How to make machines curious to Learn?**


### AL algorithm:

1. A relatively large unlabeled dataset is gathered.
2. A domain expert labels a few positive examples in the dataset.

<img src=https://imgur.com/BHi6GRx.png width="400">

3. A classifier is trained on labeled samples.
4. The classifier is applied to the rest of the corpus.

#<img src=https://imgur.com/ZPya6mX.png width="400">



5. Few most “useful” examples are selected (e.g., that increase classification performance).
6. The examples labeled by the expert are added to the training set.

<img src=https://imgur.com/yCSp1kU.png width="400">

7. Goto 3.


<img src=https://imgur.com/smisThj.png width="400">

<img src=https://imgur.com/NuD954f.png width="400">


Full Loop

<img src=https://modal-python.readthedocs.io/en/latest/_images/active-learning.png width="400">

## Measures of uncertainty

#### Low Confidence
The simplest measure of uncertainty is ***low confidence*** defined by slide 22. $$x_{LC}=1-P(\hat{y}|x)$$

Where x is the instance to be predicted and $\hat{y}$ is the certainty of the most likely prediction. 

For example, if you have three classes [car, tree, plane] and your network's classification probabilities are [0.1 | 0.2 | 0.7] the most likely class according to your classifier is plane with 70%. We take 100% and subtract the highest probability to get a 0.3 uncertainty score. The samples with the lowest uncertainty score are then queried.


Exercise (from exam):

*   Which of these samples should be labeld by an expert who uses the ***low confidence*** metric?
    *   a) [0.5 | 0.3 | 0.2]
    *   b) [1.0 | 0.0 | 0.0]
    *   c) [0.0 | 1.0 | 0.0]
    *   d) [0.4 | 0.3 | 0.3 ]
    *   e) [0.5 | 0.0 | 0.5 ]
    






In [ ]:
proba = torch.tensor([
                    [0.5, 0.3, 0.2],  # a) Position 0
                    [1.0, 0.0, 0.0],  # b) Position 1
                    [0.0, 1.0, 0.0],  # c) Position 2
                    [0.4, 0.3, 0.3],  # d) Position 3
                    [0.5, 0.0, 0.5]]) # e) position 4

the corresponding uncertainties are:

In [ ]:
highest_confidence = proba.max(axis=1)[0]
print(f"the highest confidences for a class are : {highest_confidence.data} \n")
subtract = 1-highest_confidence
print(f"Subtract from one {subtract} \n")
print(f"the lowest confidence is on position {subtract.argmax()} with value {subtract.max()}")

In the example above, the most uncertain sample is d) because 0.4 was the highest certainty.

In [ ]:
## Code for plotting no understanding necessary

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from itertools import product

n_res = 100
p1, p2 = np.meshgrid(np.linspace(0, 1, n_res), np.linspace(0, 1, n_res))
p3 = np.maximum(1 - p1 - p2, 0)

uncertainty = 1 - np.maximum.reduce([p1, p2, p3])

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 10))
    plt.contourf(p1, p2, uncertainty*((p1+p2) < 1), 100)
    plt.title('Lowest Confidence')

#### Smallest Margin Confidence
The measure of uncertainty ***smallest margin*** is defined by slide 24. 

$$ x_{M}=\underset{x}{\operatorname{arg min}} P(\hat{y_1}|x) - P(\hat{y_2}|x)$$

We are searching for the instances where the probability of the two most certain classes is very similar to each other. This means that the neural net can't decide between two classes.

For example, if you have three classes [car, tree, plane] and your network's classification probabilities are [0.1 | 0.2 | 0.7] the most likely class according to your classifier is plane with 70% and the second highest is tree with 20%. We subtract 0.2 from 0.7 and get a 0.5 uncertainty score. The samples with the lowest confidence scores are then queried.



Exercise (from exam):

*   Which of these samples should be labeld by an expert, who is following the ***smallest margin*** metric?
    *   a) [0.5 | 0.3 | 0.2]
    *   b) [1.0 | 0.0 | 0.0]
    *   c) [0.0 | 1.0 | 0.0]
    *   d) [0.4 | 0.3 | 0.3 ]
    *   e) [0.5 | 0.0 | 0.5 ]
    






In [ ]:
proba = torch.tensor([
                    [0.5, 0.3, 0.2],  # a) Position 0
                    [1.0, 0.0, 0.0],  # b) Position 1
                    [0.0, 1.0, 0.0],  # c) Position 2
                    [0.4, 0.3, 0.3],  # d) Position 3
                    [0.5, 0.0, 0.5]]) # e) position 4

In [ ]:
two_highest_probabilities = torch.topk(proba, k=2, axis=1)[0]
print(f"top two classes \n{two_highest_probabilities}")

subtraction = torch.abs(two_highest_probabilities[:,0] - two_highest_probabilities[:,1])
print(f"\nsubtract the highest class with the second highest class \n{subtraction}" )
print(f"\nThe lowest margin is on Position {torch.argmin(subtraction)} with value {torch.min(subtraction)}")

In [ ]:
# just for visualization
proba = np.vstack((p1.ravel(), p2.ravel(), p3.ravel())).T

part = np.partition(-proba, 1, axis=1)
margin = - part[:, 0] + part[:, 1]

margin = margin.reshape(p1.shape)

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 10))
    plt.contourf(p1, p2, (1-margin)*((p1+p2) < 1), 100)
    plt.title('Classification margin for ternary classification')



Entropy

The measure of uncertainty ***highest entropy*** is defined by slide 24. 

$$ x_{E}=\underset{x}{\operatorname{arg max}} \sum_y -P(y|x) log P(y|x)$$

For example, if you have three classes [car, tree, plane] and your network's classification probabilities are [0.1 | 0.2 | 0.7].
Then we calculate 

$$ -0.1 \log(0.1) - 0.2 \log(0.2) - 0.7 \log(0.7) = 0.8$$ 

which is the uncertainty score. We are searching for the instances with the highest entropy


Exercise (not in exam but it could be):

*   Which of these samples should be labeld by an expert, who is following the ***highest entropy*** metric?
    *   a) [0.5 | 0.3 | 0.2]
    *   b) [1.0 | 0.0 | 0.0]
    *   c) [0.0 | 1.0 | 0.0]
    *   d) [0.4 | 0.3 | 0.3 ]
    *   e) [0.5 | 0.0 | 0.5 ]
    

In [ ]:
from scipy.stats import entropy # let scipy calculate our entropy
proba = torch.tensor([
                    [0.5, 0.3, 0.2],  # a) Position 0
                    [1.0, 0.0, 0.0],  # b) Position 1
                    [0.0, 1.0, 0.0],  # c) Position 2
                    [0.4, 0.3, 0.3],  # d) Position 3
                    [0.5, 0.0, 0.5]]) # e) position 4

ent = entropy(proba.T)
print(f"Entropy of all instances is {ent} \nmaximum Entropy is {ent.max()} \nat Position {ent.argmax()}")

In [ ]:
# just for visualization
proba = np.vstack((p1.ravel(), p2.ravel(), p3.ravel())).T
entr = entropy(proba.T).reshape(p1.shape)

with plt.style.context('seaborn-white'):
    plt.figure(figsize=(10, 10))
    plt.contourf(p1, p2, entr*((p1+p2) < 1), 100)
    plt.title('Entropy for ternary classification')


## Active Learning on CIFAR10
We will be using `modAL`, a modular active learning framework for Python. modAL is built on top of scikit-learn, but you can also use TensorFlow, Keras or PyTorch models.

To start, you will import some pytorch packages:

In [ ]:
## Packages for Dirctories
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

## Dataset preparation
**CIFAR10** is a dataset which contains 60K low resolution RGB pictures. 50k images are normally used for training and 10k images are used for validation

In [ ]:
# Load and prepare the CIFAR dataset.

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=50000,
                                            shuffle=False, num_workers=0,
                                            worker_init_fn=seed_worker,
                                            generator=g)

validset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
validloader = torch.utils.data.DataLoader(validset, batch_size=10000,
                                            shuffle=False, num_workers=0,
                                            worker_init_fn=seed_worker,
                                            generator=g)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Verify the data
To verify that the dataset looks correct, let's plot the first 25 images from the training set and display the class name below each image.

In [ ]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(trainloader)
# we are loading the full dataset into RAM. This might cause OOM Errors here.
images, labels = dataiter.next()
vis_imgs = images[:6]
vis_labels = labels[:6]

# show images
imshow(torchvision.utils.make_grid(vis_imgs))
# print labels
print(' '.join(f'{classes[vis_labels[j]]:5s}' for j in range(6)))

# convert training images in numpy
images = images.detach().numpy()
labels = labels.detach().numpy()

# also get the validation data
valid_images, valid_labels = iter(validloader).next()
valid_images = valid_images.detach().numpy()
valid_labels = valid_labels.detach().numpy()

We will train the network with a small part of dataset to make the training for the exercise faster

In [ ]:
print (f"The full training dataset contains {len(trainset)} elements")
print (f"The full validation dataset contains {len(validset)} elements")


### Pool-Based Sampling
In pool-based sampling the machine has access to a large number of examples  (in our example 1000 samples) and samples from a pool (50000 samples) based on “informativeness.” Informativeness is quantified based on a user-selected metric.

In [ ]:
# assemble initial data
n_initial = 1000

# take 1000 random images and labels from our training dataset
initial_idx = np.random.choice(range(len(images)), size=n_initial, replace=False)
x_initial = images[initial_idx]
y_initial = labels[initial_idx]

# delete the initial images from our pool
# we are using our pool to find samples, that have a high uncertainty score and "label" them
x_pool = np.delete(images, initial_idx, axis=0)
y_pool = np.delete(labels, initial_idx, axis=0)

print(f"our initial dataset containes {len(x_initial)} images ")
print(f"our remaining 'unlabeled' dataset contains {len(x_pool)} images")


### Create the Model

We are wrapping our PyTorch model with Skorch to make it compatible with our Active Learning Library  

In [ ]:
%pip install skorch

Build CNN, define lossfunction and set optimizer. 
Wrap model with skorch

In [ ]:
from skorch import NeuralNetClassifier

# create very small neural net
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
device = "cuda" if torch.cuda.is_available() else "cpu"
print("we are working on", device)

model = NeuralNetClassifier(net,
                            criterion=nn.CrossEntropyLoss(),
                            optimizer=optim.Adam,
                            optimizer__weight_decay=0.001,
                            warm_start=True,
                            lr = 0.001,
                            train_split=None,
                            verbose=1,
                            device=device)

### Active Learning
Suppose that you can query the label of an unlabelled instance, but it costs you a lot. Which one would you choose? By querying an instance in the uncertain region, surely you obtain more information than querying by random. 

The key components of any workflow are the **model** you choose, the **informativeness** measure you use and the **query** strategy you apply to request labels. modAL was designed with modularity, flexibility and extensibility in mind. With using the scikit-learn API, it allows you to rapidly create active learning workflows. You have the freedom to seamlessly integrate scikit-learn, TensorFlow/Keras or PyTorch models into your algorithm and easily tailor your custom query strategies and uncertainty measures.

In [ ]:
%pip install modAL

In [ ]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling, margin_sampling, entropy_sampling

Here we initialize our Active Learner. It takes as input:

*   Our skorch neural net
*   A query strategy:
    * lowest confidence strategy is called classification uncertainty
    * smallest margin strategy is called classification margin
    * highest entropy is called classification entropy
* the initial training data
* how many epochs we want to train on this original data

In [ ]:
# initialize ActiveLearner
learner = ActiveLearner(
    estimator=model,
    query_strategy=uncertainty_sampling,
    # the following argumets will be used to train the model
    X_training=x_initial, 
    y_training=y_initial,
    epochs=10,
)

print("accuracy on validation",learner.score(valid_images, valid_labels)*100,"%")
print("acc score on training", learner.score(x_initial, y_initial)*100,"%")

Actively query the human user

In [ ]:
n_queries = 5

uncertain_images = []
uncertain_labels = []
for i in range(n_queries):

    # learner.query(x_pool) calculates the uncertainty for each element in our
    # pool and returns the element with the highest uncertainty
    query_idx, query_inst = learner.query(x_pool)
    # visualize image with highest uncertainty
    imshow(torchvision.utils.make_grid(torch.tensor(query_inst)))
    # what is our neural net currently predicting?
    predictions = net(torch.tensor(query_inst).to(device))
    # the predicted class:
    class_idx = torch.argmax(predictions).cpu().numpy()

    # print the results
    print("the network prediction: ", classes[class_idx])
    print("the class in our dataset", classes[y_pool[query_idx][0]])
    print("class percentages: ", F.softmax(predictions, dim=1).detach().cpu().numpy() * 100, "%")

    print("Which class is this?")
    for j in range(len(classes)):
        print(f"{j}: {classes[j]}")
    
    y_new = np.array([int(input())], dtype=int)
    uncertain_images.append(query_inst[0])
    uncertain_labels.append(y_new[0])
    # delete the sample with the high uncertainty from the pool
    x_pool, y_pool = np.delete(x_pool, query_idx, axis=0), np.delete(y_pool, query_idx, axis=0)

# this adds the high uncertainty images with labels to our initial dataset and retrains
learner.teach(np.array(uncertain_images), np.array(uncertain_labels), epochs=1)
print("accuracy on validation",learner.score(valid_images, valid_labels)*100,"%")


#Let's cheat: 
Our dataset already contains the correct labels.
Let's take them instead of asking a human to label them.

(It makes no sense to use active learning on an already labeled dataset. But it's easier to show the advantages).

In [ ]:
n_queries = 10

for i in range(n_queries):
    print('Query no. %d' % (i + 1))
    
    # take the 500 most uncertain images 
    query_idx, query_inst = learner.query(x_pool,n_instances=1000)
    learner.teach(X=x_pool[query_idx], y=y_pool[query_idx])
    x_pool = np.delete(x_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)
    print("accuracy on validation",learner.score(valid_images, valid_labels)*100,"%")

We had 1,000 initial images and added 24,000 images with active learning. With these 25000 labes we got a validation accuracy of ~47.5%

How high is our validation accuracy on 25,000 random images?



In [ ]:
random_model = NeuralNetClassifier(Net(),
                            criterion=nn.CrossEntropyLoss(),
                            optimizer=optim.Adam,
                            optimizer__weight_decay=0.001,
                            lr = 0.001,
                            train_split=None,
                            verbose=1,
                            device=device)
random_idx = np.random.choice(range(len(images)), size=25000, replace=False)
x_random = images[initial_idx]
y_random = labels[initial_idx]

random_model.fit(x_random, y_random, epochs=100)
print("accuracy on training", random_model.score(x_random, y_random))
print("accuracy on validation",random_model.score(valid_images, valid_labels)*100,"%")



This is less than 47.5%. So if labeling is expensive, it's better to use active learning